# SAMOS Spectroscopy - Apply Wavelength Calibration to All Slits

## Status: ⚠️ In Development

This notebook is being updated to use the new `samos` package structure.

**What this notebook does:**
1. Loads wavelength solution from reference slit (notebook 05)
2. Applies wavelength calibration to all slits
3. Creates wavelength-calibrated 1D spectra
4. Creates wavelength-calibrated 2D spectra
5. Saves as standard FITS format with WCS headers
6. Outputs `spec1d_*.fits` and `spec2d_*.fits` for visualization

**Prerequisites:**
- Run Notebooks 01, 04, and 05 first
- Wavelength solution saved as `wl_poly_coefficients_*.txt`

---

## Current Workaround

**Option 1: Use the old notebook**
- The original algorithms are preserved in `06_apply_calibration_OLD.ipynb`
- They still work, just use old `Class_SAMOS` structure
- You can adapt the code manually if needed

**Option 2: Use external tools**
- **PypeIt**: Complete spectroscopic reduction
  - https://pypeit.readthedocs.io/
  - Handles wavelength calibration and application
  - Standard FITS output format
  - Install: `pip install pypeit`

**Option 3: Wait for full update**
- This notebook will be fully updated soon
- Check back or request priority

---

## What Needs to Be Done

To complete this notebook, we need to:

1. Load wavelength solution coefficients
2. Apply to all extracted slits
3. Create proper WCS headers for FITS files
4. Handle variable wavelength solutions per slit (if needed)
5. Resample to constant wavelength grid (optional)

---

## Simplified Workflow (What You Can Do Now)

Here's a simplified version of the key steps using available tools:

In [ ]:
# Standard imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from astropy.io import fits
from astropy import units as u
from specutils import Spectrum1D

# SAMOS package
from samos.utils import io, display

print("✓ Packages loaded")

## Configuration

In [ ]:
# Target information
target_name = "ABELL3120"
target_mode = "SAMI_manual_Mask_T00_Low_Red"

# Working directory (where spec_*.fits files are)
analysis_top_directory = "/Users/nestrada/Documents/SAMOS/SAMOS_REDUCED"
working_directory = os.path.join(analysis_top_directory, target_name, target_mode)

os.chdir(working_directory)
print(f"Working directory: {working_directory}")

# Reference slit (from notebook 05)
reference_slit = 10
print(f"Reference slit: {reference_slit}")

# Find all spec files
spec_files = sorted(Path('.').glob('spec_*.fits'))
print(f"Found {len(spec_files)} slit files to calibrate")

## Important Note

The cells below show the **concept** of what needs to be done for each step.

For a **complete, production-ready implementation**, either:
1. Use the algorithms from `06_apply_calibration_OLD.ipynb`
2. Use PypeIt (recommended for publication-quality results)
3. Wait for the full `samos.spectroscopy.wavelength_cal` module

---

## Step 1: Load Wavelength Solution

In [ ]:
# Load wavelength solution from reference slit
wl_coeffs_file = f'wl_poly_coefficients_{reference_slit:03d}.txt'

if not Path(wl_coeffs_file).exists():
    print(f"⚠️ Wavelength solution not found: {wl_coeffs_file}")
    print("Please run notebook 05 first to create wavelength solution.")
else:
    # Load coefficients
    coeffs = np.loadtxt(wl_coeffs_file)
    wavelength_poly = np.poly1d(coeffs)
    
    print(f"✓ Wavelength solution loaded from: {wl_coeffs_file}")
    print(f"Polynomial degree: {len(coeffs) - 1}")
    print(f"Coefficients: {coeffs}")
    
    # Test the solution
    test_pixels = np.array([0, 1000, 2000, 3000, 4000])
    test_wavelengths = wavelength_poly(test_pixels)
    
    print("\nTest wavelengths:")
    for pix, wl in zip(test_pixels, test_wavelengths):
        print(f"  Pixel {pix:4d} → {wl:.1f} Å")

## Step 2: Apply Calibration to All Slits

Loop through all slit files and apply the wavelength solution.

**Note**: This example assumes the same wavelength solution works for all slits.
For higher precision, you may need slit-specific solutions (see notebook 05_OLD).

In [ ]:
# Process all slits
n_processed = 0
n_skipped = 0

for spec_file in spec_files:
    # Extract slit number from filename
    slit_number = int(spec_file.stem.split('_')[1])
    
    try:
        # Load slit data
        hdul = fits.open(spec_file)
        
        # Check if SPEC-2D extension exists (from notebook 04)
        if 'SPEC-2D' not in [ext.name for ext in hdul]:
            print(f"⚠️ Skipping {spec_file.name}: No SPEC-2D extension (run notebook 04 first)")
            hdul.close()
            n_skipped += 1
            continue
        
        # Extract 2D spectrum
        spec_2d = hdul['SPEC-2D'].data
        dx = spec_2d.shape[1]
        
        # Apply wavelength solution
        x_pixels = np.arange(dx)
        wavelengths = wavelength_poly(x_pixels)
        
        # Extract 1D spectrum (collapse spatially)
        # Use central rows (adjust as needed)
        center_y = spec_2d.shape[0] // 2
        extraction_width = 4  # Extract ±4 pixels around center
        
        y_start = max(0, center_y - extraction_width)
        y_end = min(spec_2d.shape[0], center_y + extraction_width)
        
        spectrum_1d = spec_2d[y_start:y_end, :].mean(axis=0)
        
        hdul.close()
        
        # Trim to valid wavelength range (e.g., 5800-10000 Å)
        wl_min, wl_max = 5800, 10000
        valid_range = (wavelengths >= wl_min) & (wavelengths <= wl_max)
        
        wavelengths_trimmed = wavelengths[valid_range]
        spectrum_1d_trimmed = spectrum_1d[valid_range]
        spec_2d_trimmed = spec_2d[:, valid_range]
        
        # Create Spectrum1D objects with units
        spec1d = Spectrum1D(
            spectral_axis=wavelengths_trimmed * u.AA,
            flux=spectrum_1d_trimmed * u.adu
        )
        
        spec2d = Spectrum1D(
            spectral_axis=wavelengths_trimmed * u.AA,
            flux=spec_2d_trimmed * u.adu
        )
        
        # Save calibrated spectra
        spec1d_file = f'spec1d_{slit_number:03d}.fits'
        spec2d_file = f'spec2d_{slit_number:03d}.fits'
        
        spec1d.write(spec1d_file, overwrite=True)
        spec2d.write(spec2d_file, overwrite=True)
        
        n_processed += 1
        
        # Progress update every 10 slits
        if n_processed % 10 == 0:
            print(f"Processed {n_processed} slits...")
    
    except Exception as e:
        print(f"⚠️ Error processing {spec_file.name}: {e}")
        n_skipped += 1
        continue

print(f"\n✓ Calibration complete!")
print(f"  Processed: {n_processed} slits")
print(f"  Skipped: {n_skipped} slits")
print(f"\nOutput files:")
print(f"  spec1d_*.fits - 1D wavelength-calibrated spectra")
print(f"  spec2d_*.fits - 2D wavelength-calibrated spectra")

## Step 3: Verify Output

Load and display a few calibrated spectra to verify the wavelength calibration.

In [ ]:
# Find all spec1d files
spec1d_files = sorted(Path('.').glob('spec1d_*.fits'))

if len(spec1d_files) == 0:
    print("⚠️ No spec1d files found. Run the calibration step above first.")
else:
    print(f"Found {len(spec1d_files)} wavelength-calibrated 1D spectra")
    
    # Plot first 5 spectra
    n_plot = min(5, len(spec1d_files))
    
    fig, axes = plt.subplots(n_plot, 1, figsize=(16, 3*n_plot))
    if n_plot == 1:
        axes = [axes]
    
    for i, spec_file in enumerate(spec1d_files[:n_plot]):
        spec1d = Spectrum1D.read(spec_file)
        
        axes[i].plot(spec1d.spectral_axis, spec1d.flux)
        axes[i].set_xlabel('Wavelength (Å)')
        axes[i].set_ylabel('Flux (ADU)')
        axes[i].set_title(f'{spec_file.name}')
        axes[i].grid(alpha=0.3)
        
        # Show wavelength range
        wl_min = spec1d.spectral_axis.min().value
        wl_max = spec1d.spectral_axis.max().value
        axes[i].text(0.02, 0.95, f'{wl_min:.0f} - {wl_max:.0f} Å',
                    transform=axes[i].transAxes, 
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    print("\n✓ Wavelength calibration verified")

## Step 4: Summary Statistics

In [ ]:
# Collect summary info
spec1d_files = sorted(Path('.').glob('spec1d_*.fits'))
spec2d_files = sorted(Path('.').glob('spec2d_*.fits'))

print("=" * 60)
print("WAVELENGTH CALIBRATION SUMMARY")
print("=" * 60)
print(f"\nTarget: {target_name}")
print(f"Mode: {target_mode}")
print(f"Reference slit: {reference_slit}")
print(f"\nOutput files:")
print(f"  1D spectra: {len(spec1d_files)} files")
print(f"  2D spectra: {len(spec2d_files)} files")

if len(spec1d_files) > 0:
    # Load one spectrum to show wavelength range
    test_spec = Spectrum1D.read(spec1d_files[0])
    wl_min = test_spec.spectral_axis.min().value
    wl_max = test_spec.spectral_axis.max().value
    n_pixels = len(test_spec.spectral_axis)
    dispersion = (wl_max - wl_min) / n_pixels
    
    print(f"\nWavelength range: {wl_min:.1f} - {wl_max:.1f} Å")
    print(f"Number of pixels: {n_pixels}")
    print(f"Mean dispersion: {dispersion:.3f} Å/pixel")

print(f"\nWorking directory:")
print(f"  {working_directory}")
print("\n" + "=" * 60)
print("\n✓ Ready for notebook 07 (visualization)!")
print("\nNext steps:")
print("  1. Run notebook 07 to visualize spectra with jdaviz")
print("  2. Check spectral features and emission/absorption lines")
print("  3. Begin scientific analysis of your spectra!")

---

## Advanced: Slit-Specific Wavelength Solutions

For higher precision, you may want to derive wavelength solutions for each slit individually.

This handles small variations in wavelength solution across the detector.

**See `06_apply_calibration_OLD.ipynb` for the full implementation.**

In [ ]:
# Example: Load slit-specific wavelength solution if it exists
# 
# This would be used in the processing loop above:
# 
# for slit_number in range(n_slits):
#     # Try to load slit-specific solution
#     slit_wl_file = f'wl_poly_coefficients_{slit_number:03d}.txt'
#     
#     if Path(slit_wl_file).exists():
#         # Use slit-specific solution
#         coeffs = np.loadtxt(slit_wl_file)
#     else:
#         # Fall back to reference solution
#         coeffs = np.loadtxt(wl_coeffs_file)
#     
#     wavelength_poly = np.poly1d(coeffs)
#     # ... continue with calibration

print("This is a conceptual example.")
print("See the OLD notebook for the full implementation.")

---

## Using PypeIt (Recommended Alternative)

For complete wavelength calibration workflow:

```python
# Install PypeIt
# pip install pypeit

from pypeit import wavecalib
from pypeit.core import flexure

# PypeIt provides:
# - Automatic wavelength solution for all slits
# - Flexure correction
# - Wavelength resampling to constant grid
# - Standard FITS output with WCS headers
# - Quality assessment
```

See: https://pypeit.readthedocs.io/

---

## Full Algorithm Reference

The complete, tested algorithms are in `06_apply_calibration_OLD.ipynb`.

Key features of the full implementation:
- Cross-correlation between slits for wavelength alignment
- Slit-specific wavelength solutions
- Proper WCS header creation
- Text file output (wavelength, flux) for external tools
- Batch processing with error handling

To use the old algorithms:
1. Open `06_apply_calibration_OLD.ipynb`
2. The calibration application code will still work
3. Update imports to use new `samos` package where applicable

---

## Status and Recommendations

**Current status**: Simplified workflow provided above

**For full functionality**:
1. Use `06_apply_calibration_OLD.ipynb` algorithms
2. Or use PypeIt for professional results (recommended)
3. Or wait for full integration in `samos` package

**Next step**: Notebook 07 for visualization!

---

Last updated: December 15, 2025